In [94]:
import Wheel_Controller as wc
import time

tkr = "NVDA"
while(True):
  if (wc.trade_active()): #If trade active wait
    time.sleep(3600)
  elif(wc.puts_cycle(tkr)): #If no trades and still own 100 stocks
    wc.sell_puts(tkr)
  else:
    wc.sell_calls(tkr)



In [106]:
# SELL PUTS
import requests
import pandas as pd
import math

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "PKX38EM432GQ49SYV8SA",
    "APCA-API-SECRET-KEY": "lXwrv1l82QIQvvwakv6hWQ8s6JYSmEEW9MAu7vX8"
}



date = "2024-07-05" # ONE WEEK INTO FURUTRE YYYY-MM-DD
tkr = "NVDA"
def get_current_price(tkr):
    price_url = f"https://data.alpaca.markets/v2/stocks/bars/latest?symbols={tkr}"
    price_data = requests.get(price_url, headers=headers).json()
    return float(price_data['bars'][f"{tkr}"]['c'])

curr_price = get_current_price(tkr) # USE AS UPPER BOUND
strike_price = math.floor(curr_price - (math.ceil(curr_price * .01))) # USE AS LOWER BOUND

puts_url = f"https://paper-api.alpaca.markets/v2/options/contracts?underlying_symbol={tkr}&expiration_date_lte={date}&type=put&strike_price_gte={strike_price}&strike_price_lte={curr_price}&limit=20"
puts_resp = requests.get(puts_url, headers=headers)
print(type(puts_resp.status_code))
print(puts_resp.status_code)
puts_data = puts_resp.json()['option_contracts']
contract_id = puts_data[len(puts_data)-1]['symbol']




<class 'int'>
200


In [113]:
# TRADE ACTIVE
import requests

url = "https://paper-api.alpaca.markets/v2"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "PKX38EM432GQ49SYV8SA",
    "APCA-API-SECRET-KEY": "lXwrv1l82QIQvvwakv6hWQ8s6JYSmEEW9MAu7vX8"
}
def trades_active():
    response = requests.get(url+"/positions", headers=headers)
    if (response.status_code == 200):
      positions = response.json()
      for position in positions:
         if (len(position['symbol']) > 4):
            return True
      return False
    else:
       #time.sleep(60)
       trades_active()
    print(positions)

trades_active()


[{'asset_id': '4ce9353c-66d1-46c2-898f-fce867ab0247', 'symbol': 'NVDA', 'exchange': 'NASDAQ', 'asset_class': 'us_equity', 'asset_marginable': True, 'qty': '100', 'avg_entry_price': '122.98', 'side': 'long', 'market_value': '12354', 'cost_basis': '12298', 'unrealized_pl': '56', 'unrealized_plpc': '0.0045535859489348', 'unrealized_intraday_pl': '0', 'unrealized_intraday_plpc': '0', 'current_price': '123.54', 'lastday_price': '123.54', 'change_today': '0', 'qty_available': '100'}]


In [101]:

# GET next exp day
from datetime import datetime, timedelta
dt = datetime.now()
td = timedelta(days=7)
# your calculated date
my_date = dt + td

def get_exp_date():
    dt = datetime.now()
    td = timedelta(days=7)
    # your calculated date
    next_exp = dt + td
    return next_exp.strftime("%Y")+"-"+next_exp.strftime("%m")+"-"+next_exp.strftime("%d")
print (get_exp_date())



2024-07-03


In [ ]:
# SUBMIT ORDER FOR SELL PUTS
# id = get_options_id() // not_implemented
sell_puts_payload = {
  "symbol": puts_data[len(puts_data)-1]['symbol'], # EX OF id: "AAPL231201P00175000"
  "qty": "1",
  "side": "sell",
  "type": "market",
  "time_in_force": "day"
}

orders_url = "https://paper-api.alpaca.markets/v2/orders"
response = requests.post(orders_url, json=sell_puts_payload, headers=headers)


In [114]:

puts_url = "https://paper-api.alpaca.markets/v2/options/contracts?underlying_symbols=NVDA&expiration_date=2024-07-05&type=put&strike_price_gte=124&strike_price_lte=126&limit=20"
puts_data = requests.get(puts_url, headers=headers).json()
print(puts_data)

{'option_contracts': [{'id': '8584f300-590d-4072-b6f3-9516294c5847', 'symbol': 'NVDA240705P00124000', 'name': 'NVDA Jul 05 2024 124 Put', 'status': 'active', 'tradable': True, 'expiration_date': '2024-07-05', 'root_symbol': 'NVDA', 'underlying_symbol': 'NVDA', 'underlying_asset_id': '4ce9353c-66d1-46c2-898f-fce867ab0247', 'type': 'put', 'style': 'american', 'strike_price': '124', 'multiplier': '100', 'size': '100', 'open_interest': '9688', 'open_interest_date': '2024-06-28', 'close_price': '3.03', 'close_price_date': '2024-06-28'}, {'id': '498f8623-4821-4bfa-b341-d9e47c24954f', 'symbol': 'NVDA240705P00125000', 'name': 'NVDA Jul 05 2024 125 Put', 'status': 'active', 'tradable': True, 'expiration_date': '2024-07-05', 'root_symbol': 'NVDA', 'underlying_symbol': 'NVDA', 'underlying_asset_id': '4ce9353c-66d1-46c2-898f-fce867ab0247', 'type': 'put', 'style': 'american', 'strike_price': '125', 'multiplier': '100', 'size': '100', 'open_interest': '15925', 'open_interest_date': '2024-06-28', 'cl

In [88]:

import math
headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "PKX38EM432GQ49SYV8SA",
    "APCA-API-SECRET-KEY": "lXwrv1l82QIQvvwakv6hWQ8s6JYSmEEW9MAu7vX8"
}

def get_current_price(tkr):
    price_url = f"https://data.alpaca.markets/v2/stocks/bars/latest?symbols={tkr}"
    price_data = requests.get(price_url, headers=headers).json()
    return float(price_data['bars'][f"{tkr}"]['c'])
curr_price = get_current_price("NVDA")
strike_price = math.floor(curr_price - (math.ceil(curr_price * .01)))
print(strike_price)


123
